In [1]:
import pandas as pd
import numpy as np

In [2]:
%load_ext autoreload
%autoreload 2

from utils.archive2 import WGS84Coordinate
from utils.archive import MapboxApiClient, MapboxRoutingProfile, MapboxRouteEnum
from typing import List

In [3]:
df = pd.read_parquet("hilo_full_patterns.parquet")

In [4]:
df.columns

Index(['placekey', 'safegraph_place_id', 'parent_placekey',
       'parent_safegraph_place_id', 'location_name', 'safegraph_brand_ids',
       'brands', 'top_category', 'sub_category', 'category_tags', 'naics_code',
       'latitude', 'longitude', 'street_address', 'city', 'region',
       'postal_code', 'iso_country_code', 'phone_number', 'open_hours',
       'opened_on', 'closed_on', 'tracking_opened_since',
       'tracking_closed_since', 'date_range_start', 'date_range_end',
       'raw_visit_counts', 'raw_visitor_counts', 'visits_by_day',
       'visitor_home_cbgs', 'visitor_country_of_origin', 'distance_from_home',
       'median_dwell', 'bucketed_dwell_times', 'related_same_day_brand',
       'related_same_month_brand', 'popularity_by_hour', 'popularity_by_day',
       'device_type', 'poi_cbg', 'visitor_daytime_cbgs'],
      dtype='object')

In [13]:
df.head()

,placekey,safegraph_place_id,parent_placekey,parent_safegraph_place_id,location_name,safegraph_brand_ids,brands,top_category,sub_category,category_tags,...,distance_from_home,median_dwell,bucketed_dwell_times,related_same_day_brand,related_same_month_brand,popularity_by_hour,popularity_by_day,device_type,poi_cbg,visitor_daytime_cbgs
2353,zzw-223@bp4-jct-t5f,sg:a6abadf575c64c7aa2957973c322c258,None,None,YMCA,SG_BRAND_8c40d09d7a855232f6ae8762ffd5966a,YMCA,Individual and Family Services,Other Individual and Family Services,None,...,NaN,NaN,None,None,None,None,None,None,NaN,None
3153,22d-222@bp4-jct-f75,sg:e170329441704027b328f7603705729f,228-222@bp4-jct-hkf,sg:d4f8d56cb61345e6b33a5a1382a503d7,Happy Valley Seafood Restaurant,None,None,Restaurants and Other Eating Places,Full-Service Restaurants,Chinese Food,...,9178.0,14.0,"{""<5"":10,""5-10"":23,""11-20"":11,""21-60"":18,""61-1...",{},"{""CVS"":57,""Target"":49,""7-Eleven"":35,""Walmart"":...","[1,1,0,1,0,0,2,8,6,7,10,13,15,13,9,11,7,3,0,1,...","{""Monday"":13,""Tuesday"":9,""Wednesday"":11,""Thurs...","{""android"":21,""ios"":24}",1.500102e+11,"{""150010210131"":4,""150010207012"":4,""1500102050..."
3422,226-222@bp4-jct-zpv,sg:f6130790a4c7440592d15fe1a58888a6,None,None,Two Ladies Kitchen,None,None,Bakeries and Tortilla Manufacturing,Retail Bakeries,"Dessert,Bakery",...,NaN,NaN,None,None,None,None,None,None,NaN,None
4179,zzy-222@bp4-jcw-xkf,sg:2dc8ca526096494b8694c5ba704704f1,zzw-222@bp4-jfh-47q,sg:5bcbf8781047448ab852bfc69c5c515e,Wailoa River State Recreation Area,None,None,"Museums, Historical Sites, and Similar Institu...",Nature Parks and Other Similar Institutions,None,...,4596.0,47.0,"{""<5"":0,""5-10"":5,""11-20"":15,""21-60"":13,""61-120...","{""Panda Express"":50,""Target"":48}","{""CVS"":51,""Target"":49,""Walmart"":44,""Minit Stop...","[0,1,0,0,0,0,0,13,17,12,18,14,6,4,12,12,8,1,1,...","{""Monday"":6,""Tuesday"":6,""Wednesday"":14,""Thursd...","{""android"":5,""ios"":5}",1.500102e+11,"{""150010211064"":4,""150010205003"":4}"
9436,zzw-223@bp4-jcv-2ff,sg:9ce49fae5cba4828990d087fb1eaaf75,None,None,Hawaii Probation Office,None,None,"Justice, Public Order, and Safety Activities",Courts,None,...,8829.0,63.5,"{""<5"":4,""5-10"":41,""11-20"":20,""21-60"":34,""61-12...","{""7-Eleven"":19,""76"":17,""Starbucks"":11,""Walmart...","{""CVS"":52,""Walmart"":36,""Burger King"":35,""The H...","[0,0,0,0,0,0,21,64,60,67,62,67,59,45,37,33,30,...","{""Monday"":42,""Tuesday"":35,""Wednesday"":56,""Thur...","{""android"":24,""ios"":31}",1.500102e+11,"{""150010205005"":5,""150010204003"":4,""1500102170..."


In [16]:
# group on place_id, location name, lat long 
# sum on visit counts
grouped_df = df.groupby(['safegraph_place_id', 'location_name', 'latitude', 'longitude'], as_index=False)['raw_visit_counts'].sum()

print(grouped_df)

                       safegraph_place_id  \
0     sg:003c91fcb4b94ac388c2591f3c2447e0   
1     sg:003feb8df3804f93955c52b1b09a819f   
2     sg:004bbb2a0079462a935a0a0220e2b4b2   
3     sg:00aea7e992024e468f1485a59ccbe4a6   
4     sg:0107d7dfb69c48c69d9a5cef22303061   
...                                   ...   
1289  sg:fefdd677eac743fbacdc98cf9a4f84ac   
1290  sg:ff1254a8fc5843da80a6c842d88f52f9   
1291  sg:ff2d34072c114835aee3ee11b688c5c4   
1292  sg:ff675c511ad246b7a55705bc7ea14bc9   
1293  sg:ffd5f48fd0ce49449d2a74bee549105c   

                             location_name   latitude   longitude  \
0                     Blue Hawaii Antiques  19.723642 -155.086409   
1                King's Vietnamese Cuisine  19.722987 -155.085033   
2                             Sunglass Hut  19.697561 -155.063257   
3     Nate Gaddis Elite Pacific Properties  19.726001 -155.088203   
4                 First Hawaiian Bank Hilo  19.711231 -155.076523   
...                                    ...   

In [18]:
# Extract latitude and longitude columns
valid_df = grouped_df[['latitude', 'longitude']].dropna()
coordinates = [
    WGS84Coordinate(lat=row['latitude'], lon=row['longitude'])
    for _, row in valid_df.iterrows()
]

In [19]:
len(coordinates)

1294

In [20]:
# Create a walking routing profile
walking_profile = MapboxRoutingProfile(name=MapboxRouteEnum.WALKING)

# Initialize the Mapbox API client
api_client = MapboxApiClient()

In [21]:
# Generate the distance matrix
distance_matrix = api_client.make_distance_matrix(coordinates, walking_profile)

# Convert the matrix to a DataFrame
distance_df = pd.DataFrame(distance_matrix, index=df.index, columns=df.index)

print(distance_df)

  0%|          | 0/1294 [00:00<?, ?it/s]

  2%|▏         | 32/1294 [40:24<26:33:24, 75.76s/it]


KeyboardInterrupt: 

In [25]:
# Define a function to compute the weighted distance matrix

def compute_weighted_distance_matrix(api_client: MapboxApiClient, coords: List[WGS84Coordinate], profile: MapboxRoutingProfile, visit_counts: List[int]) -> pd.DataFrame:
    # Compute the distance matrix
    raw_distance_matrix = api_client.make_distance_matrix(coords, profile)

    # Convert to a numpy array 
    distance_matrix = np.array(raw_distance_matrix)
    
    # Normalize travel times
    max_distance = distance_matrix.max()
    normalized_matrix = distance_matrix / max_distance
    
    # Apply location weights
    visit_counts_array = np.array(visit_counts)
    location_weights = visit_counts_array / visit_counts_array.sum()
    
    # Create Weighted Distance Matrix
    weighted_distance_matrix = normalized_matrix * location_weights[:, np.newaxis]
    
    # Convert back to a DataFrame
    weighted_distance_df = pd.DataFrame(weighted_distance_matrix, index=range(len(coords)), columns=range(len(coords)))
    
    return weighted_distance_df

In [29]:
# Testing

valid_df = df[['latitude', 'longitude', 'raw_visit_counts']].dropna()
coordinates = [
    WGS84Coordinate(lat=row['latitude'], lon=row['longitude'])
    for _, row in valid_df.iterrows()
]
visit_counts = valid_df['raw_visit_counts'].tolist()

walking_profile = MapboxRoutingProfile(name=MapboxRouteEnum.WALKING)
api_client = MapboxApiClient()

weighted_distance_df = compute_weighted_distance_matrix(api_client, coordinates, walking_profile, visit_counts)

  0%|          | 0/31447 [00:00<?, ?it/s]


RuntimeError: Error fetching distances from the Mapbox API. The server returned a "422-Unknown" status code with the text: "{"message":"Too many coordinates; maximum number of coordinates is 25.","code":"InvalidInput"}".

In [ ]:
# Call the solver

In [ ]:
# Plot results on a folium map